Installations

In [ ]:
%pip install pandas numpy

Imports

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2e}'.format

Perf files

In [ ]:

bench_gen_perf_file = "./data/benchGenPerf.csv"
bench_gen_perf = pd.read_csv(bench_gen_perf_file)

llvm_test_suite_perf_file = "./data/llvmTestSuitePerf.csv"
llvm_test_suite_perf = pd.read_csv(llvm_test_suite_perf_file)

spec_2017_perf_file = "./data/spec2017Perf.csv"
spec_2017_perf = pd.read_csv(spec_2017_perf_file)

Perf events

In [ ]:
llvm_spec_intersection = llvm_test_suite_perf.columns.intersection(spec_2017_perf.columns)
perf_events = bench_gen_perf.columns.intersection(llvm_spec_intersection)

Filtering dataframes

In [ ]:
llvm_test_suite_perf_filtered = llvm_test_suite_perf[perf_events]
spec_2017_perf_filtered = spec_2017_perf[perf_events]
bench_gen_perf_filtered = bench_gen_perf[perf_events]
bench_gen_perf_filtered = bench_gen_perf_filtered.apply(pd.to_numeric, errors='coerce')

Centroids

In [ ]:

llvm_test_suite_centroids = llvm_test_suite_perf_filtered.mean()
spec_2017_centroids = spec_2017_perf_filtered.mean()
bench_gen_centroids = bench_gen_perf_filtered.mean()

Distances

In [ ]:
llvm_distances_to_centroid = np.abs(llvm_test_suite_perf_filtered - llvm_test_suite_centroids)
llvm_average_distance_to_centroid = llvm_distances_to_centroid.mean()
llvm_distances_to_centroid_relative = llvm_distances_to_centroid / llvm_test_suite_centroids
llvm_average_distance_to_centroid_relative = llvm_distances_to_centroid_relative.mean()

spec_2017_distances_to_centroid = np.abs(spec_2017_perf_filtered - spec_2017_centroids)
spec_2017_average_distance_to_centroid = spec_2017_distances_to_centroid.mean()
spec_2017_distances_to_centroid_relative = spec_2017_distances_to_centroid / spec_2017_centroids
spec_2017_average_distance_to_centroid_relative = spec_2017_distances_to_centroid_relative.mean()

bench_gen_distances_to_centroid = np.abs(bench_gen_perf_filtered - bench_gen_centroids)
bench_gen_average_distance_to_centroid = bench_gen_distances_to_centroid.mean()
bench_gen_distances_to_centroid_relative = bench_gen_distances_to_centroid / bench_gen_centroids
bench_gen_average_distance_to_centroid_relative = bench_gen_distances_to_centroid_relative.mean()

bench_gen_distances_to_llvm_centroid = np.abs(bench_gen_perf_filtered - llvm_test_suite_centroids)
bench_gen_average_distance_to_llvm_centroid = bench_gen_distances_to_llvm_centroid.mean()
bench_gen_distances_to_llvm_centroid_relative = bench_gen_distances_to_llvm_centroid / llvm_test_suite_centroids
bench_gen_average_distance_to_llvm_centroid_relative = bench_gen_distances_to_llvm_centroid_relative.mean()

bench_gen_distances_to_spec_centroid = np.abs(bench_gen_perf_filtered - spec_2017_centroids)
bench_gen_average_distance_to_spec_centroid = bench_gen_distances_to_spec_centroid.mean()
bench_gen_distances_to_spec_centroid_relative = bench_gen_distances_to_spec_centroid / spec_2017_centroids
bench_gen_average_distance_to_spec_centroid_relative = bench_gen_distances_to_spec_centroid_relative.mean()

LLVM test suite results

In [ ]:
print("LLVM average distance to centroid:")
print(llvm_average_distance_to_centroid)
print("\n###################################################################################\n")
print("LLVM average relative distance to centroid:")
print(llvm_average_distance_to_centroid_relative)

SPEC 2017 results

In [ ]:
print("SPEC 2017 average distance to centroid:")
print(spec_2017_average_distance_to_centroid)
print("\n###################################################################################\n")
print("SPEC 2017 average relative distance to centroid:")
print(spec_2017_average_distance_to_centroid_relative)

BenchGen results

In [ ]:
print("BenchGen average distance to centroid:")
print(bench_gen_average_distance_to_centroid)
print("\n###################################################################################\n")
print("BenchGen average relative distance to centroid:")
print(bench_gen_average_distance_to_centroid_relative)

BenchGen X LLVM test suite results

In [ ]:
print("BenchGen average distance to LLVM centroid:")
print(bench_gen_average_distance_to_llvm_centroid)
print("\n###################################################################################\n")
print("BenchGen average relative distance to LLVM centroid:")
print(bench_gen_average_distance_to_llvm_centroid_relative)

BenchGen X SPEC 2017 results

In [ ]:
print("BenchGen average distance to SPEC 2017 centroid:")
print(bench_gen_average_distance_to_spec_centroid)
print("\n###################################################################################\n")
print("BenchGen average relative distance to SPEC 2017 centroid:")
print(bench_gen_average_distance_to_spec_centroid_relative)

Generate CSV

In [59]:
row0 = pd.Index(perf_events, name="Performance Events")
row1 = pd.Index(llvm_average_distance_to_centroid, name="LLVM average distance to centroid")
row2 = pd.Index(spec_2017_average_distance_to_centroid, name="SPEC 2017 average distance to centroid")
row3 = pd.Index(bench_gen_average_distance_to_centroid, name="BenchGen average distance to centroid")
row4 = pd.Index(bench_gen_average_distance_to_llvm_centroid, name="BenchGen average distance to LLVM centroid")
row5 = pd.Index(bench_gen_average_distance_to_spec_centroid, name="BenchGen average distance to SPEC 2017 centroid")

row6 = pd.Index(llvm_average_distance_to_centroid_relative, name="LLVM average relative distance to centroid")
row7 = pd.Index(spec_2017_average_distance_to_centroid_relative, name="SPEC 2017 average relative distance to centroid")
row8 = pd.Index(bench_gen_average_distance_to_centroid_relative, name="BenchGen average relative distance to centroid")
row9 = pd.Index(bench_gen_average_distance_to_llvm_centroid_relative, name="BenchGen average relative distance to LLVM centroid")
row10 = pd.Index(bench_gen_average_distance_to_spec_centroid_relative, name="BenchGen average relative distance to SPEC 2017 centroid")

s0 = pd.Series(row0, name=row0.name)
s1 = pd.Series(row1, name=row1.name)
s2 = pd.Series(row2, name=row2.name)
s3 = pd.Series(row3, name=row3.name)
s4 = pd.Series(row4, name=row4.name)
s5 = pd.Series(row5, name=row5.name)
s6 = pd.Series(row6, name=row6.name)
s7 = pd.Series(row7, name=row7.name)
s8 = pd.Series(row8, name=row8.name)
s9 = pd.Series(row9, name=row9.name)
s10 = pd.Series(row10, name=row10.name)

df = pd.DataFrame({s0.name : s0, s1.name: s1, s2.name: s2, s3.name: s3, s4.name: s4, s5.name: s5, s6.name: s6, s7.name: s7, s8.name: s8, s9.name: s9, s10.name: s10})

df = df.transpose()

df.to_csv("./results/analysisResult.csv")